# Recursive Path Replacement in JSON Checkpoint File

In [1]:
# Changing '/kaggle/working/preprocessed_frames/train/0_3' to '/kaggle/input/preprocessed_frames/train/0_3' in extraction_checkpoint['metadata']

In [2]:
def update_paths_recursive(obj, old_path, new_path):
    if isinstance(obj, dict):
        return {k: update_paths_recursive(v, old_path, new_path) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [update_paths_recursive(i, old_path, new_path) for i in obj]
    elif isinstance(obj, str):
        return obj.replace(old_path, new_path)
    else:
        return obj

In [3]:
import json

# Load the original JSON
with open('/kaggle/input/meld-extracted-video-frames/extraction_checkpoint.json', 'r') as file:
    data = json.load(file)

# Replace all "/kaggle/working" with "/kaggle/input"
updated_data = update_paths_recursive(data, "/kaggle/working", "/kaggle/input/meld-extracted-video-frames")

# Optional: Save to file
with open('/kaggle/working/extraction_checkpoint_updated.json', 'w') as file:
    json.dump(updated_data, file, indent=2)

---

# Convert Video Frame Images from BGR to RGB Using OpenCV

In [4]:
import os
import cv2
from tqdm import tqdm

# Define input and output base directories
input_base = '/kaggle/input/meld-extracted-video-frames/preprocessed_frames'
output_base = '/kaggle/working/preprocessed_frames'

# Collect all JPG file paths
jpg_files = []
for root, dirs, files in os.walk(input_base):
    for file in files:
        if file.lower().endswith('.jpg'):
            jpg_files.append(os.path.join(root, file))

# Process each file with progress indication
for input_path in tqdm(jpg_files, desc="Converting Images"):
    # Generate the corresponding output path
    relative_path = os.path.relpath(input_path, input_base)
    output_path = os.path.join(output_base, relative_path)
    
    # Ensure output directory exists
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    
    # Read image using OpenCV (BGR format)
    bgr_frame = cv2.imread(input_path)
    
    if bgr_frame is not None:
        # Convert BGR to RGB by reversing channels
        rgb_frame = bgr_frame[..., ::-1]
        
        # Save the converted image (OpenCV will write it as BGR, but the conversion makes it correct for RGB viewers)
        cv2.imwrite(output_path, rgb_frame)
    else:
        print(f"Error: Could not read image {input_path}")

print("Processing complete!")

Converting Images: 100%|██████████| 438592/438592 [51:03<00:00, 143.18it/s]

Processing complete!
